In [1]:
import os
import sys
import pathlib
import shutil
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras import regularizers
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import seaborn as sn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def create_label_dir(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe contains labels for each image and path to the directory

    contains the unlabeled dataset to rebuild directory into labeled subdirectories.

    Returns all the label and number of classes in the dataset.

    Keyword arguments:

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train)
    """
    class_names = np.sort(df['label'].unique())
    number_of_classes = len(class_names)

    if not os.path.exists(dir):
        return class_names, number_of_classes

    for class_name in class_names:
        subdir = pathlib.Path(os.path.join(dir, class_name))
        if subdir.exists():
            continue
        else:
            subdir.mkdir()
    
    return class_names, number_of_classes

def sort_data(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe to move each unlabeled image to the correct label's subdirectory.

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train) 
    """
    if not os.path.exists(dir):
        return
    
    unlabeled_dir = os.path.join(dir, 'train')

    for image_dir in [str(img) for img in list(pathlib.Path(unlabeled_dir).glob('*.png'))]:
        id = int(image_dir.removeprefix(unlabeled_dir).removesuffix('.png'))
        label = df['label'][id - 1]
        dest_path = os.path.join(dir, label, str(id) + '.png')
        shutil.move(image_dir, dest_path)

In [3]:
label_df = pd.read_csv('../gdsc-ai-challenge/trainLabels.csv')

class_names, number_of_classes = create_label_dir(label_df)
sort_data(label_df)

In [4]:
def split_dataset(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1):
    """Split the dataset into three subsets: train, validation (dev) and test set.

    Returns three tuples, containing each subset with its size.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    train_split -- percentage to split into train set (default to 0.8)

    val_split -- percentage to split into validation set (default to 0.1)

    test_split -- percentage to split into test set (default to 0.1)
    """
    assert (train_split + test_split + val_split) == 1
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return (train_ds, train_size), (val_ds, val_size), (test_ds, ds_size - val_size - train_size)

def configure(ds, ds_size, batch_size=32, shuffle=False, augment=False):
    """Configure the given dataset for better performance (by caching, prefetching and then batching the dataset)

    and perform preprocessing to the images in the given dataset.

    Returns the optimized dataset.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    batch_size -- size of each batch (default to 32)

    shuffle -- whether to shuffle the dataset (default to False)

    augment -- whether to perform data augmentation to the dataset (default to False)
    """

    AUTOTUNE = tf.data.AUTOTUNE
    rescale = keras.layers.Rescaling(1.0/255)
    data_augmentation = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(0.05, fill_mode='nearest')
    ])

    ds = ds.map(lambda x, y: (rescale(x), y),
                num_parallel_calls=AUTOTUNE)

    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(buffer_size=int(ds_size * 0.6))
    
    ds = ds.batch(batch_size)

    if augment:
        with tf.device('/cpu:0'):
            #only perform data augmentation on train set
            ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                                    num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [5]:
ds = tf.keras.utils.image_dataset_from_directory(
    '../gdsc-ai-challenge/train',
    color_mode='grayscale',
    batch_size=None,
    image_size=(32,32),
    seed=42
)

ds_size = ds.cardinality().numpy()

Found 50000 files belonging to 10 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-23 04:04:57.846331: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-23 04:04:57.846469: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
(train_ds, train_size), (val_ds, val_size), (test_ds, test_size) = \
split_dataset(ds, ds_size, train_split=0.7, val_split=0.2, test_split=0.1)

train_ds = configure(train_ds, train_size, augment=True, shuffle=True)
val_ds = configure(val_ds, val_size)
test_ds = configure(test_ds, test_size)

In [7]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
        #adding first convolutional layer    
        keras.layers.Input((32,32,1))
    ])
    blocks = hp.Int('blocks', min_value=3, max_value=4)

    for b in range(blocks):
        for i in range(hp.Int('block_{}_layers'.format(b), 1, 3)):
            num_filter = 16
            if b == 0:
                if i == 0:
                    num_filter = 16
                else:
                    num_filter = hp.get('conv_0_{}_filter'.format(i - 1))
            else:
                if i == 0:
                    num_filter = hp.get('conv_{}_{}_filter'.format(b - 1, hp.get('block_{}_layers'.format(b - 1)) - 1))
                else:
                    num_filter = hp.get('conv_{}_{}_filter'.format(b, i - 1))

            model.add(
                keras.layers.Conv2D(
                    #adding filter 
                    filters=hp.Int('conv_{}_{}_filter'.format(b, i), min_value=num_filter, max_value=512, step=16),
                    #adding filter size or kernel size
                    kernel_size=hp.Choice('conv_{}_{}_kernel'.format(b, i), values = [1,3]),
                    #activation function
                    activation='elu', padding='same',
                    kernel_regularizer=keras.regularizers.l2(hp.Float('kernel_{}_regularizer'.format(i), min_value=1e-5, max_value=1e-4, step=1e-5)),
                    activity_regularizer=keras.regularizers.l2(hp.Float('weight_{}_regularizer'.format(i), min_value=1e-5, max_value=1e-4, step=1e-5))
                )
            )
            model.add(keras.layers.BatchNormalization())

        if b < blocks - 1:
            model.add(keras.layers.MaxPooling2D((2,2)))
        else:
            model_type = hp.Choice("model_type", ["mlp", "cnn"])
            if model_type == "mlp":
                with hp.conditional_scope("model_type", ["mlp"]):
                    model.add(keras.layers.Flatten())

                    for d in range(hp.Int('dense_layers', 2, 3)):
                        num_units = 16
                        if d > 0:
                            num_units = hp.get('dense_{}_units'.format(d - 1))

                        model.add(
                            keras.layers.Dense(
                                units=hp.Int('dense_{}_units'.format(d), min_value=num_units, max_value=64, step=16), 
                                activation='elu'
                            )
                        )
                        dropout = hp.Boolean('dropout')
                        if dropout:
                            model.add(keras.layers.Dropout(
                                hp.Float('dropout_{}_layer'.format(d), min_value=0.3, max_value=0.5, step=0.05)
                            ))
            if model_type == "cnn":
                with hp.conditional_scope("model_type", ["cnn"]):
                    model.add(keras.layers.GlobalAveragePooling2D())
                    dropout = hp.Boolean('dropout')
                    if dropout:
                        model.add(keras.layers.Dropout(
                            hp.Float('dropout_1_layer', min_value=0.2, max_value=0.5, step=0.05)
                        ))

    keras.layers.Dense(10, activation='softmax')
    
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
              loss='sparse_categorical_crossentropy',
              metrics=[
                  tf.keras.losses.SparseCategoricalCrossentropy(),
                  'accuracy'])
    return model

In [8]:
tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=20, directory='trial_model', project_name='hyperband_model_1')
tuner.search(train_ds, epochs=20, validation_data=val_ds)

Trial 9 Complete [00h 14m 10s]
val_accuracy: 0.005400000140070915

Best val_accuracy So Far: 0.29170000553131104
Total elapsed time: 04h 21m 21s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
4                 |4                 |blocks
2                 |2                 |block_0_layers
112               |64                |conv_0_0_filter
3                 |1                 |conv_0_0_kernel
2e-05             |7e-05             |kernel_0_regularizer
3e-05             |2e-05             |weight_0_regularizer
1                 |3                 |block_1_layers
336               |304               |conv_1_0_filter
1                 |1                 |conv_1_0_kernel
2                 |2                 |block_2_layers
128               |384               |conv_2_0_filter
1                 |1                 |conv_2_0_kernel
mlp               |cnn               |model_type
3                 |None              |dense_layers
144               |None     

Traceback (most recent call last):
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/var/folders/vt/09r1h38x3fz6tx9dnyq1869w0000gn/T/ipykernel_16899/1299149078.py", line 59, in build_model
    hp.Int('dropout_{}_layer'.format(d), min_value=0.3, max_value=0.5, step=0.05)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 838, in Int
    hp = Int(
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 258, in __init__
    self.max_value = _check_int(max_value, arg="max_value")
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-pack

Invalid model 0/5


Traceback (most recent call last):
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/var/folders/vt/09r1h38x3fz6tx9dnyq1869w0000gn/T/ipykernel_16899/1299149078.py", line 59, in build_model
    hp.Int('dropout_{}_layer'.format(d), min_value=0.3, max_value=0.5, step=0.05)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 838, in Int
    hp = Int(
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 258, in __init__
    self.max_value = _check_int(max_value, arg="max_value")
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-pack

Invalid model 1/5


...

Traceback (most recent call last):
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/var/folders/vt/09r1h38x3fz6tx9dnyq1869w0000gn/T/ipykernel_16899/1299149078.py", line 59, in build_model
    hp.Int('dropout_{}_layer'.format(d), min_value=0.3, max_value=0.5, step=0.05)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 838, in Int
    hp = Int(
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 258, in __init__
    self.max_value = _check_int(max_value, arg="max_value")
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-pack

Invalid model 4/5


Traceback (most recent call last):
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/var/folders/vt/09r1h38x3fz6tx9dnyq1869w0000gn/T/ipykernel_16899/1299149078.py", line 59, in build_model
    hp.Int('dropout_{}_layer'.format(d), min_value=0.3, max_value=0.5, step=0.05)
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 838, in Int
    hp = Int(
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras_tuner/engine/hyperparameters.py", line 258, in __init__
    self.max_value = _check_int(max_value, arg="max_value")
  File "/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-pack

Invalid model 5/5


RuntimeError: Too many failed attempts to build model.

In [ ]:
model=tuner.get_best_models(num_models=2)[0]
#summary of best model
model.summary()